# RNA-Seq from scratch - Kallisto

Notes on the example data and experiment

## Organizing our inputs

Kallisto requires just a few simple things to run 

- A reference transcriptome
- FastQ files (your sequence data)

Since we already have this data available on the computers you are connecting to, let's find, inspect, and organize it

## Organizing files and directories

This notebook is running the bash shell. We could run all of these commands from the terminal, and if you run these on a terminal on your own after the workshop the commands will be similar (the locations of your files may be different)

### Get the working directory and set the locations of files

First, let's go to our home directory. If you ever get lost there is no place like home:

In [55]:
cd

Let's see the contents of our home directory

In [56]:
ls

bioinfosummer-rna-seq-notebooks  kallisto-rnaseq-jupyter  tutorial-data
-i                               notebooks


Another way of specifying the home directory will be to use the `$HOME` shell variable

In [57]:
ls $HOME

bioinfosummer-rna-seq-notebooks  kallisto-rnaseq-jupyter  tutorial-data
-i                               notebooks


All of the data we need for our bulk RNA-Seq experiment should be in the `tutorial-data` folder; let's inspect its contents:

In [58]:
ls $HOME/tutorial-data

data           kallisto_bulk_rna-seq_ouputs  kallisto-single-cell
kallisto-bulk  kallisto_sc_rna-seq_ouputs    processed_files


For this experiment, all of our input data files will be in the `kallisto-bulk/data` folder: 

In [59]:
ls $HOME/tutorial-data/kallisto-bulk/data

fastq_files  hnpc_SRA_accessions.txt  study_design  transcriptomes


The `fastq_files` directory contains data generated from the example experiment

In [60]:
ls $HOME/tutorial-data/kallisto-bulk/data/fastq_files

paired-end  single-end


Let's see which of the files were from paired-end sequenced libraries, and those that were single-end sequenced. A file in the `study_design` folder hass this information:

In [61]:
cat $HOME/tutorial-data/kallisto-bulk/data/study_design/zikadesignmatrix.csv

﻿,sample,condition,read,fragments,Instrument,LoadDate
1,SRR3191542,mock,paired,7927777,Illumina MiSeq,2016-02-26
2,SRR3191543,mock,paired,7391076,Illumina MiSeq,2016-02-26
3,SRR3191544,zika,paired,7361527,Illumina MiSeq,2016-02-26
4,SRR3191545,zika,paired,7621347,Illumina MiSeq,2016-02-26
5,SRR3194428,mock,single,72983243,NextSeq 500,2016-02-29
6,SRR3194429,mock,single,94729809,NextSeq 500,2016-02-29
7,SRR3194430,zika,single,71055823,NextSeq 500,2016-02-29
8,SRR3194431,zika,single,66528035,NextSeq 500,2016-02-29

Samples `SRR3194428`, `SRR3194429`, `SRR3194430`, and `SRR3194431` are single-end sequenced, so we will move the coresponding fastq files into their own directories:

In [17]:
mkdir $HOME/tutorial-data/kallisto-bulk/data/fastq_files/single-end ./tutorial-data/kallisto-bulk/data/fastq_files/paired-end 

We now have two new directories

In [36]:
ls $HOME/tutorial-data/kallisto-bulk/data/fastq_files

paired-end  single-end


Let's change into this directory so our commands don't too long

In [22]:
cd $HOME/tutorial-data/kallisto-bulk/data/fastq_files/

Now we can move the desired files

In [23]:
mv SRR3194428_1.fastq.gz SRR3194429_1.fastq.gz SRR3194430_1.fastq.gz SRR3194431_1.fastq.gz -t ./single-end

and we can move the paired-end files as well

In [24]:
mv *.fastq.gz ./paired-end

mv: cannot move 'paired-end' to a subdirectory of itself, './paired-end/paired-end'
mv: cannot stat '.fastq.gz': No such file or directory


: 1

Now we can see that the contents of both folders contain the appropriate sequences

In [34]:
ls -R

.:
paired-end  single-end

./paired-end:
SRR3191541_1.fastq.gz  SRR3191543_2.fastq.gz  SRR3191545_1.fastq.gz
SRR3191541_2.fastq.gz  SRR3191544_1.fastq.gz  SRR3191545_2.fastq.gz
SRR3191543_1.fastq.gz  SRR3191544_2.fastq.gz

./single-end:
SRR3194428_1.fastq.gz  SRR3194430_1.fastq.gz
SRR3194429_1.fastq.gz  SRR3194431_1.fastq.gz


If you remember in the previous exercise, we have already done some quality control on these files. Rather than repeat this let's move those QC files into directories here. Let's put them in a more convient place so that we don't mix raw data with processed data:

In [37]:
ls $HOME/tutorial-data/kallisto-bulk/analyses/fastp

qc_fastp_paired-end  qc_fastp_single-end  reports


let's change into this directory and organize our data:

In [83]:
cd $HOME/tutorial-data/kallisto-bulk/analyses/fastp

In [84]:
mkdir qc_fastp_paired-end qc_fastp_single-end

In [85]:
mv fastp_SRR3194428_1.fastq.gz fastp_SRR3194429_1.fastq.gz fastp_SRR3194430_1.fastq.gz fastp_SRR3194431_1.fastq.gz -t ./qc_fastp_single-end

In [86]:
mv *.fastq.gz ./qc_fastp_paired-end

In [38]:
ls -R

.:
fastp_SRR3194428_1.fastq.gz  fastp_SRR3194430_1.fastq.gz
fastp_SRR3194429_1.fastq.gz  fastp_SRR3194431_1.fastq.gz


### Obtain reference transcriptome

We obtained the human reference transcriptome data from [Ensemble](https://uswest.ensembl.org/Homo_sapiens/Info/Index). Specifically, we want the set of cDNAs available from the [ftp site](ftp://ftp.ensembl.org/pub/release-94/fasta/homo_sapiens/cdna/). Those data are available here:

In [1]:
ls $HOME/tutorial-data/kallisto-bulk/data/transcriptomes

Homo_sapiens.GRCh38.cdna.all.fa.gz  Readme.md


### Using Kallisto to index the transcriptome

We will now use Kallisto's indexing function to prepare the transcriptome for analysis. First let's organize our files:

In [64]:
mkdir $HOME/tutorial-data/kallisto-bulk/indicies

mkdir: cannot create directory ‘/home/tutorial-user/tutorial-data/kallisto-bulk/indicies’: File exists


: 1

In [6]:
ls $HOME/tutorial-data/kallisto-bulk/

analyses  data  indicies  processed


Let's change into the indicies directory where we will want to keep the index of our human transcriptome:

In [7]:
cd $HOME/tutorial-data/kallisto-bulk/indicies

Now we are finally ready to use kallisto. Let's check that Kallisto is installed, check it's version, and get a little help:

In [3]:
kallisto

kallisto 0.44.0

Usage: kallisto <CMD> [arguments] ..

Where <CMD> can be one of:

    index         Builds a kallisto index 
    quant         Runs the quantification algorithm 
    bus           Generate BUS files for single cell data 
    pseudo        Runs the pseudoalignment step 
    merge         Merges several batch runs 
    h5dump        Converts HDF5-formatted results to plaintext
    inspect       Inspects and gives information about an index
    version       Prints version information
    cite          Prints citation information

Running kallisto <CMD> without arguments prints usage information for <CMD>



: 1

Next run the indexing command (5-7 minutes). This prepares the transcriptome so that we can peudoalign reads to it. 

In [8]:
kallisto index --index="human_GRCh38_transcriptome_index" $HOME/tutorial-data/kallisto-bulk/data/transcriptomes/Homo_sapiens.GRCh38.cdna.all.fa.gz


[build] loading fasta file /home/tutorial-user/tutorial-data/kallisto-bulk/data/transcriptomes/Homo_sapiens.GRCh38.cdna.all.fa.gz
[build] k-mer length: 31
[build] warning: clipped off poly-A tail (longer than 10)
        from 1471 target sequences
[build] warning: replaced 5 non-ACGUT characters in the input sequence
        with pseudorandom nucleotides
[build] counting k-mers ... done.
[build] building target de Bruijn graph ...  done 
[build] creating equivalence classes ...  done
[build] target de Bruijn graph has 1118780 contigs and contains 108619921 k-mers 



We now have a transcriptome index which can now be used for pseudoalignment. As long as we intend to use this version of the transcriptome, we can use this index for all our future Kallisto experiments - no need to index again.

In [9]:
ls

human_GRCh38_transcriptome_index


### Quantify reads

In this final step, we will run Kallisto on all of our files to quantify the reads. We will create a directory and then write some bash shell for loops to run kallisto independently on each read file (single-end) or pair of read files (paired-end). 

In [15]:
mkdir $HOME/tutorial-data/kallisto-bulk/analyses/kallisto-quantification

In [16]:
cd $HOME/tutorial-data/kallisto-bulk/analyses/kallisto-quantification

All instructions for the commands we are using are in the Kallisto manual: https://pachterlab.github.io/kallisto/manual. 


MOAR




First, let's do the analysis of the single-end reads, based on the parameters use in the paper

In [24]:
cd $HOME/tutorial-data/kallisto-bulk/analyses/fastp/qc_fastp_single-end
for file in *.fastq.gz; do output="${file%.*.*}"_quant; kallisto quant\
 --threads=4\
 --single\
 --index=$HOME/tutorial-data/kallisto-bulk/indicies/human_GRCh38_transcriptome_index\
 --bootstrap-samples=100\
 --fragment-length=187\
 --sd=70\
 --output-dir=$output\
 $file; done


[quant] fragment length distribution is truncated gaussian with mean = 187, sd = 70
[index] k-mer length: 31
[index] number of targets: 187,626
[index] number of k-mers: 108,619,921
[index] number of equivalence classes: 752,021
[quant] running in single-end mode
[quant] will process file 1: fastp_SRR3194428_1.fastq.gz
[quant] finding pseudoalignments for the reads ... done
[quant] processed 71,471,164 reads, 55,641,327 reads pseudoaligned
[   em] quantifying the abundances ... done
[   em] the Expectation-Maximization algorithm ran for 1,514 rounds
[bstrp] number of EM bootstraps complete: 100


[quant] fragment length distribution is truncated gaussian with mean = 187, sd = 70
[index] k-mer length: 31
[index] number of targets: 187,626
[index] number of k-mers: 108,619,921
[index] number of equivalence classes: 752,021
[quant] running in single-end mode
[quant] will process file 1: fastp_SRR3194429_1.fastq.gz
[quant] finding pseudoalignments for the reads ... done
[quant] processed 

Now we should have four folders containing results of our quantification of the single-end reads:

In [25]:
ls $HOME/tutorial-data/kallisto-bulk/analyses/fastp/qc_fastp_single-end

fastp_SRR3194428_1.fastq.gz  fastp_SRR3194430_1.fastq.gz
fastp_SRR3194428_1_quant     fastp_SRR3194430_1_quant
fastp_SRR3194429_1.fastq.gz  fastp_SRR3194431_1.fastq.gz
fastp_SRR3194429_1_quant     fastp_SRR3194431_1_quant


Let's move the folders to our analyses folder:

In [26]:
mv $HOME/tutorial-data/kallisto-bulk/analyses/fastp/qc_fastp_single-end/*/ $HOME/tutorial-data/kallisto-bulk/analyses/kallisto-quantification

In [39]:
cd $HOME/tutorial-data/kallisto-bulk/analyses/fastp/qc_fastp_paired-end
for file in *_1.fastq.gz; do re1input=$file;\
 re2input=fastp_$(echo $file|cut -f2 -d _)_2.fastq.gz;\
 output="${file%.*.*}"_quant;\
 output=fastp_$(echo $file|cut -f2 -d _)_quant;\
 kallisto quant\
 --threads=4\
 --index=$HOME/tutorial-data/kallisto-bulk/indicies/human_GRCh38_transcriptome_index\
 --bootstrap-samples=100\
 --output-dir=$output\
 $re1input $re2input;\
 done


[quant] fragment length distribution will be estimated from the data
[index] k-mer length: 31
[index] number of targets: 187,626
[index] number of k-mers: 108,619,921
[index] number of equivalence classes: 752,021
[quant] running in paired-end mode
[quant] will process pair 1: fastp_SRR3191542_1.fastq.gz
                             fastp_SRR3191542_2.fastq.gz
[bstrp] number of EM bootstraps complete: 100


[quant] fragment length distribution will be estimated from the data
[index] k-mer length: 31
[index] number of targets: 187,626
[index] number of k-mers: 108,619,921
[index] number of equivalence classes: 752,021
[quant] running in paired-end mode
[quant] will process pair 1: fastp_SRR3191543_1.fastq.gz
                             fastp_SRR3191543_2.fastq.gz
[quant] finding pseudoalignments for the reads ... done
[quant] processed 7,207,925 reads, 5,916,664 reads pseudoaligned
[quant] estimated average fragment length: 178.473
[   em] quantifying the abundances ... done
[   em] t

We now will have 4 folders for the results of our paired-end quantification

In [40]:
ls $HOME/tutorial-data/kallisto-bulk/analyses/fastp/qc_fastp_paired-end/

fastp_SRR3191542_1.fastq.gz  fastp_SRR3191544_1.fastq.gz
fastp_SRR3191542_2.fastq.gz  fastp_SRR3191544_2.fastq.gz
fastp_SRR3191542_quant       fastp_SRR3191544_quant
fastp_SRR3191543_1.fastq.gz  fastp_SRR3191545_1.fastq.gz
fastp_SRR3191543_2.fastq.gz  fastp_SRR3191545_2.fastq.gz
fastp_SRR3191543_quant       fastp_SRR3191545_quant


Let's also move these to analyses, and verify all 8 samples are there

In [41]:
mv $HOME/tutorial-data/kallisto-bulk/analyses/fastp/qc_fastp_paired-end/*/ $HOME/tutorial-data/kallisto-bulk/analyses/kallisto-quantification/

In [42]:
ls $HOME/tutorial-data/kallisto-bulk/analyses/kallisto-quantification/

fastp_SRR3191542_quant  fastp_SRR3191545_quant    fastp_SRR3194430_1_quant
fastp_SRR3191543_quant  fastp_SRR3194428_1_quant  fastp_SRR3194431_1_quant
fastp_SRR3191544_quant  fastp_SRR3194429_1_quant


### Examining Kallisto Outputs

Given the parameters we used, we expect the following outputs (as taken from the [kallisto manual](https://pachterlab.github.io/kallisto/manual)):

- **abundances.h5**: HDF5 binary file containing run info, abundance esimates, bootstrap estimates, and transcript length information length. This file can be read in by sleuth
- **abundances.tsv**: Plaintext file of the abundance estimates. It does not contains bootstrap estimates (but these can be output using the `--plaintext` argument. `kallisto h5dump` can be used to output an HDF5 file to plaintext. The first line contains a header for each column, including estimated counts, TPM, effective length.
- **run_info.json**:json file containing information about the run

Let's look at the first few lines of the abundences file in `SRR3191541_quant`:


In [43]:
head $HOME/tutorial-data/kallisto-bulk/analyses/kallisto-quantification/fastp_SRR3191542_quant/abundance.tsv

target_id	length	eff_length	est_counts	tpm
ENST00000632684.1	12	7.2	0	0
ENST00000434970.2	9	4.2	0	0
ENST00000448914.1	13	8.2	0	0
ENST00000415118.1	8	3.2	0	0
ENST00000631435.1	12	7.2	0	0
ENST00000390583.1	31	11.5882	0	0
ENST00000390577.1	37	11.931	0	0
ENST00000451044.1	17	10.5	0	0
ENST00000390578.1	31	11.5882	0	0


We can also get information about the run in the json file

In [44]:
cat $HOME/tutorial-data/kallisto-bulk/analyses/kallisto-quantification/fastp_SRR3191542_quant/run_info.json

{
	"n_targets": 187626,
	"n_bootstraps": 100,
	"n_processed": 7837636,
	"n_pseudoaligned": 6375981,
	"n_unique": 1761917,
	"p_pseudoaligned": 81.4,
	"p_unique": 22.5,
	"kallisto_version": "0.44.0",
	"index_version": 10,
	"start_time": "Wed Nov 28 00:07:35 2018",
	"call": "kallisto quant --threads=4 --index=/home/tutorial-user/tutorial-data/kallisto-bulk/indicies/human_GRCh38_transcriptome_index --bootstrap-samples=100 --output-dir=fastp_SRR3191542_quant fastp_SRR3191542_1.fastq.gz fastp_SRR3191542_2.fastq.gz"
}
